### This notebook takes export annotation files from labelbox and locally saves the corresponding images and corresponding coco.json file
#### Also allows visualization of the resulting annotations

In [2]:
import os
import sys
import json

geladas_root = os.path.dirname(os.path.dirname(os.getcwd()))
local_paths_file = os.path.join(geladas_root, 'local-paths.json')
with open(paths_file, "r") as json_file:
    local_paths = json.load(json_file)

sys.path.append(local_paths['general_dection_path'])
from functions.labelbox_processing import labelbox_to_coco
from functions.json_functions import create_train_val_split

In [4]:
# Path to where annotated images should be saved
images_save_folder = os.path.join(local_paths['annotations_folder'], 
                                  "annotated_images")
# Path to where annotation cocojson file should be saved
coco_json_folder = local_paths['annotations_folder']
# name of the exported file from labelbox
labelbox_json_name = "export-2022-01-05T15_54_50.050Z-test.json"
labelbox_json_file =  os.path.join(local_paths['labelbox_folder'], 
                                   labelbox_json_name)

In [12]:
# Info to include in the coco annotation file being created
dataset_description = "gelada dataset"
dataset_date = "Jan.05.2021"

# The name of the created json file will be based on the name of labelbox file
coco_json_name = f"coco_{os.path.basename(labelbox_json_file)}"
coco_json_file = os.path.join(coco_json_folder, coco_json_name)

In [13]:
# Save the images and created coco format json file that corresponds to the
# labelbox.json file being used
labelbox_to_coco(labelbox_json_file, 
                 coco_json_file,
                 images_save_folder, 
                 description=dataset_description, 
                 date=dataset_date,
                 overwrite=False, 
                 verbose=True
                )   

1 gelada
2 human
192 annotations from 16 images saved.


### Code below can be used to visualize saved annotations

In [1]:
import cv2
import matplotlib.pyplot as plt

from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer

In [4]:
register_coco_instances("visualize", {}, 
                        coco_json_file, 
                        images_save_folder
                       )
data_metadata = MetadataCatalog.get("visualize")
data_dicts = DatasetCatalog.get("visualize")

In [5]:
for d in data_dicts[:2]:
    img = cv2.imread(d["file_name"])
    plt.figure(figsize=(10, 10))
    plt.imshow(img[..., ::-1])
    visualizer = Visualizer(img[:, :, ::-1], 
                            metadata=data_metadata, 
                            scale=1.0
                           )
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(10, 10))
    plt.imshow(vis.get_image())